In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime


In [1]:
# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "6645db6eeb5aad0007a6e4b6"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][1]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


NameError: name 'dash' is not defined

In [ ]:
# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",  # Replace with actual sensor IDs
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',            # or 'csv' if you prefer CSV format
    'from-date': '2024-05-18T09:00:00Z',
    #'to-date': datetime.now(),
    'download': 'true',          # This forces a download if format is 'csv'
    'outliers': 'mark',          # Optional: 'mark' or 'replace' if outlier detection is needed
    'outlier-window': 15,        # Optional: default is 15, can be 1-50
    'delimiter': 'comma'         # Optional: 'comma' or 'semicolon' for CSV
}

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    # Construct the endpoint URL
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    
    # Send the GET request to the API
    response = requests.get(endpoint, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert response data to JSON
        data = response.json()
        # Add sensor ID to each measurement
        for measurement in data:
            measurement['Id'] = sensorId
            # Check for outliers and add an 'outlier' column
            if 'outlier' in measurement:
                measurement['outlier'] = measurement['outlier']
            else:
                measurement['outlier'] = None  # If no outlier info, set as None
        # Append data to the list
        all_data.extend(data)
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')

# Convert the list of data to a DataFrame
sense_df = pd.DataFrame(all_data)
sense_df

In [ ]:
import dash
from dash import html

app = dash.Dash(__name__)

# Layout definieren
app.layout = html.Div([
    # Streifen auf der linken Seite
    html.Div(
        id='side-panel',
        style={
            'width': '25%',  # Breite des Streifens einstellen
            'position': 'fixed',  # Position fixieren
            'top': 0,  # Oben anordnen
            'left': 0,  # Links anordnen
            'bottom': 0,  # Unten anordnen
            'backgroundColor': '#f0f0f0',  # Hintergrundfarbe einstellen
            'overflowY': 'scroll'  # Scrollen ermöglichen, wenn der Inhalt überläuft
        },
        children=[
            # Inhalte des Streifens
            html.Div([
                html.H2('Umweltmonitoring SS24 Projektarbeit'),
                html.P('Hier kannst du verschiedene Informationen anzeigen lassen.'),
                # Beispielbild mit angepasster Größe
                html.Img(
                    src='https://docs.sensebox.de/images/2020-10-16-opensensemap-faq/openSenseMap_github.png',
                    style={
                        'width': '90px',  # Breite des Bildes einstellen
                        'height': '40px'  # Höhe des Bildes einstellen
                    }
                )
            ], style={'padding': '20px'})
        ]
    ),
    
    # Hauptinhalt auf der rechten Seite
    html.Div([
        html.H1('Hauptinhalt'),
        html.P('Hier befindet sich der Hauptinhalt der Seite.')
    ], style={'marginLeft': '40%', 'padding': '10px'})  # Linken Rand einstellen, um Platz für den Streifen zu machen
])

if __name__ == '__main__':
    app.run_server(debug=True)
